https://rukovodstvo.net/posts/id_599/

In [1]:
import pandas as pd

Создадим датафрейм, назначив индексом столбец ID

In [3]:
df = pd.read_csv('../data/auto.csv', engine='python', index_col='ID')
df

,CarNumber,Make_n_model,Refund,Fines,History
ID,,,,,
0,Y163O8161RUS,Ford Focus,2.0,3200.0,NaN
1,E432XX77RUS,Toyota Camry,1.0,6500.0,NaN
2,7184TT36RUS,Ford Focus,1.0,2100.0,NaN
3,X582HE161RUS,Ford Focus,2.0,2000.0,NaN
4,E34877152RUS,Ford Focus,2.0,6100.0,NaN
...,...,...,...,...,...
926,Y163O8161RUS,Ford Focus,2.0,1600.0,NaN
927,M0309X197RUS,Ford Focus,1.0,22300.0,NaN
928,O673E8197RUS,Ford Focus,2.0,600.0,NaN


Посчитаем количество наблюдений

In [4]:
df.count()

CarNumber       931
Make_n_model    931
Refund          914
Fines           869
History          82
dtype: int64

Удалим повторные записи. Уникальными считаем сочетания значений в трех заданных столбцах, между двумя одинаковыми наблюдениями нужно выбрать последнее.

In [5]:
df = df.drop_duplicates(subset=['CarNumber', 'Make_n_model', 'Fines'], keep='last')
df.count()

CarNumber       725
Make_n_model    725
Refund          713
Fines           665
History          65
dtype: int64

Проверим, сколько значений отсутствует в каждом столбце

In [6]:
df.isna().sum()

CarNumber         0
Make_n_model      0
Refund           12
Fines            60
History         660
dtype: int64

Удалим все столбцы с более чем 500 отсутствующими значениями, используя фргумент thresh

Проверим, сколько осталось пропущенных значений в каждом столбце.

In [7]:
df = df.dropna(axis=1, thresh=500)
df.isna().sum()

CarNumber        0
Make_n_model     0
Refund          12
Fines           60
dtype: int64

Заменим все пропущенные значения в столбце Refund предыдущим значением в столбце Refund.

In [8]:
df['Refund'].fillna(method='ffill', inplace=True)
df.isna().sum()

CarNumber        0
Make_n_model     0
Refund           0
Fines           60
dtype: int64

Заменим все пропущенные значения в столбце Fines средним значением этого столбца

In [9]:
df['Fines'].fillna(df['Fines'].mean(), inplace=True)
df.isna().sum()

CarNumber       0
Make_n_model    0
Refund          0
Fines           0
dtype: int64

Разделим наименование машины (Make_n_model) на марку и модель. Используем метод apply. 
Удалим столбец Make_n_model

In [10]:
df['Make'] = df['Make_n_model'].apply(lambda x: x.split()[0])
df['Model'] = df['Make_n_model'].apply(lambda x: x.split()[1] if len(x.split()) == 2 else None)
df.drop('Make_n_model', inplace=True, axis=1)
df

,CarNumber,Refund,Fines,Make,Model
ID,,,,,
0,Y163O8161RUS,2.0,3200.000000,Ford,Focus
1,E432XX77RUS,1.0,6500.000000,Toyota,Camry
2,7184TT36RUS,1.0,2100.000000,Ford,Focus
3,X582HE161RUS,2.0,2000.000000,Ford,Focus
5,92918M178RUS,1.0,5700.000000,Ford,Focus
...,...,...,...,...,...
926,Y163O8161RUS,2.0,1600.000000,Ford,Focus
927,M0309X197RUS,1.0,22300.000000,Ford,Focus
928,O673E8197RUS,2.0,600.000000,Ford,Focus


Сохраним датафрейм в файле JSON

In [12]:
df.to_json('../data/auto.json', orient='records', double_precision=1)